In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../flaming-choripan') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [2]:
from flamingchoripan.files import search_for_filedirs
from lchandler import C_

rootdir = '../../astro-lightcurves-handler/save'
filedirs = search_for_filedirs(rootdir, fext=C_.EXT_SPLIT_LIGHTCURVE)

────────────────────────────────────────────────────────────────────────────────────────────────────
found filedirs: (../../astro-lightcurves-handler/save)
(0) - ../../astro-lightcurves-handler/save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0.splcds - 5.163[mbs]
────────────────────────────────────────────────────────────────────────────────────────────────────
filedirs after searching with filters: (../../astro-lightcurves-handler/save)
(0) - ../../astro-lightcurves-handler/save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0.splcds - 5.163[mbs]
────────────────────────────────────────────────────────────────────────────────────────────────────


In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from flamingchoripan.files import load_pickle, save_pickle
from flamingchoripan.files import get_dict_from_filedir
from lchandler import C_

def load_lcdataset(filename):
    assert filename.split('.')[-1]==C_.EXT_SPLIT_LIGHTCURVE
    return load_pickle(filename)

filedir = '../../astro-lightcurves-handler/save/alerceZTFv5.1/survey-alerceZTFv5.1_bands-gr_mode-onlySNe_kfid-0.splcds'
filedir = '../../astro-lightcurves-handler/save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0.splcds'

filedic = get_dict_from_filedir(filedir)
root_folder = filedic['*rootdir*']
cfilename = filedic['*cfilename*']
lcdataset = load_lcdataset(filedir)
print(lcdataset['raw'].keys())
print(lcdataset['raw'].get_random_lcobj(False).keys())
print(lcdataset)

> loading: ../../astro-lightcurves-handler/save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0.splcds
dict_keys(['data', 'survey', 'description', 'band_names', 'class_names', 'obs_is_flux'])
dict_keys(['days', 'obs', 'obse', 'y', 'synthetic'])
LCDataset:
[raw - samples 2,047]
(*) obs_samples: 65,174 - min_len: 6 - max_dur: 538.8[days] - p50_dur: 52.9[days]
(g) obs_samples: 28,351 - min_len: 0 - max_dur: 538.8[days] - p50_dur: 38.9[days] - p50_cadence: 3.0[days]
(r) obs_samples: 36,823 - min_len: 0 - max_dur: 538.8[days] - p50_dur: 50.9[days] - p50_cadence: 3.0[days]
   |▏       | SLSN - 33/2,047 (1.61%)
   |█████▉  | SNIa - 1,534/2,047 (74.94%)
   |▍       | SNIbc - 103/2,047 (5.03%)
   |█▍      | allSNII - 377/2,047 (18.42%)
────────────────────────────────────────────────────────────────────────────────────────────────────
[raw_train - samples 1,636]
(*) obs_samples: 51,337 - min_len: 6 - max_dur: 443.9[days] - p50_dur: 52.9[days]
(g) obs_samples: 22,311 - min_len: 0

In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import flamingchoripan.files as ff

method = 'curvefit'
method = 'mcmc'
lcset_name = 'train' # hardcoded
lcset = lcdataset[lcset_name]
synth_rootdir = f'../save/{lcset.survey}/{method}'
synth_lcset = lcset.copy({})
filedirs = ff.get_filedirs(synth_rootdir, fext='synsne')
for filedir in filedirs:
#for lcobj_name in lcset.get_lcobj_names():   
    #filedir =  f'../save/{lcset.survey}/{method}/{lcobj_name}.synsne'
    d = ff.load_pickle(filedir, verbose=0)
    lcobj_name = d['lcobj_name']
    lcobj = d['lcobj']
    synth_lcset.set_lcobj(lcobj_name, lcobj)
    
    has_corrects_samples = d['has_corrects_samples'] # add ignore and those
    if has_corrects_samples: # and...
        for k,new_lcobj in enumerate(d['new_lcobjs']):
            synth_lcset.set_lcobj(f'{lcobj_name}.{k}', new_lcobj)

lcdataset.set_lcset(f'{lcset_name}_{method}', synth_lcset)
pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
print(lcdataset)

ValueError: 'SNIbc' is not in list

In [8]:
from flamingchoripan.files import load_pickle, save_pickle
from lchandler import C_

save_rootdir = f'../save/{synth_lcset.survey}'
save_filedir = f'{save_rootdir}/{cfilename}_method-{method}.{C_.EXT_SPLIT_LIGHTCURVE}'
save_pickle(save_filedir, lcdataset)

> saving: ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0_method-curvefit.splcds


'../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0_method-curvefit.splcds'